# Mecánica Pulmonar: Análisis sin Surfactante

## 1. Sin Surfactante

### Energía Total (Muy Esquemática)

La funcional de energía potencial total del sistema se expresa como:

$$\Pi_0[\chi] = \int_{\Omega_0} W(F)\,dV - \int_{\Omega_t} p_{\text{alv}}\, dV - \int_{\Omega_0} \rho_0\, \mathbf{g}\cdot \chi\, dV - \int_{\Gamma_{\text{ext},0}} \mathbf{t}_{\text{pleural}}\cdot \chi\, dA_0$$

#### Componentes de la Funcional:

| Término | Símbolo | Descripción |
|---------|---------|-------------|
| **Deformación hiperelástica** | $W(F)$ | Energía de deformación (Neo-Hooke, etc.) |
| **Presión alveolar** | $p_{\text{alv}}$ | Presión interna en los alvéolos (airway pressure) |
| **Gravedad** | $\mathbf{g}$ | Campo gravitacional |
| **Tracción pleural** | $\mathbf{t}_{\text{pleural}}$ | Tracción en la superficie externa (pleural) |

---

## 2. Configuración de Equilibrio sin Surfactante

La configuración de equilibrio $\chi_0$ se obtiene como la solución del problema variacional que minimiza $\Pi_0[\chi]$.

### Ecuación de Equilibrio en el Volumen

En el interior del tejido pulmonar $\Omega_0$, se cumple la ecuación de balance de momentum:

$$\text{Div}_X\,P(F) + \rho_0\,\mathbf{g} = \mathbf{0} \quad\text{en } \Omega_0$$

donde:
- $\text{Div}_X$ es la divergencia respecto a las coordenadas materiales $X$
- $P(F) = \frac{\partial W}{\partial F}$ es el **tensor de tensiones de Piola** (o primer tensor de esfuerzos de Piola-Kirchhoff)
- $\rho_0$ es la densidad material
- $\mathbf{g}$ es el vector aceleración gravitacional

### Condiciones de Frontera

#### En la Superficie Alveolar Interna ($\Gamma_{\text{alv},0}$)

En el borde del tejido en contacto directo con el aire alveolar, se aplica:

$$P(F)\,\mathbf{N} = -p_{\text{alv}}\, J\,F^{-T}\mathbf{N}$$

donde:
- $\mathbf{N}$ es la normal exterior en la configuración material
- $J = \det(F)$ es el Jacobiano de la deformación
- $F^{-T} = (F^{-1})^T$ es la transpuesta inversa del gradiente de deformación

**Interpretación:** Solo la presión interna alveolar actúa como tracción normal sobre el tejido. No hay contribución de energía superficial (surfactante) en esta versión simplificada.

#### Configuración de Referencia

La configuración sin surfactante ($\chi_0$) representa el **estado de equilibrio bajo cargas mecánicas** pero **sin efectos de tensión superficial**. Este es el punto de partida para posteriores análisis que incluyan la acción del surfactante pulmonar.

---

## 3. Notas Importantes

- La ausencia de surfactante implica que la única contribución energética de interfaz aire-líquido es a través de la presión alveolar
- La energía hiperelástica $W(F)$ se asume independiente de la curvatura (sin bending stiffness)
- Este modelo es válido para deformaciones moderadas donde la aproximación hiperelástica es razonable
- La gravedad y la tracción pleural proporcionan cargas externas distribuidas y concentradas respectivamente

In [41]:
from mpi4py import MPI
import numpy as np
import gmsh
from dolfinx import mesh, io
from dolfinx.cpp.mesh import CellType
import os
import tempfile
import shutil

def create_annulus_mesh(R_in, R_out, mesh_resolution=0.1, filename=None):
    """
    Crea una malla 2D de un anillo (domino circular con agujero)
    
    Parámetros:
    - R_in: Radio interno
    - R_out: Radio externo  
    - mesh_resolution: Resolución de la malla
    - filename: Nombre del archivo de salida (opcional)
    """
    
    # Usar archivo temporal si no se proporciona nombre
    if filename is None:
        temp_dir = tempfile.mkdtemp()
        filename = os.path.join(temp_dir, "annulus_mesh.xdmf")
    
    # Inicializar GMSH
    gmsh.initialize()
    gmsh.option.setNumber("General.Terminal", 0)  # Silenciar output de GMSH
    
    try:
        gmsh.model.add("annulus")
        
        # Crear geometría del anillo
        # Puntos para el círculo exterior
        outer_points = []
        outer_points.append(gmsh.model.geo.addPoint(0, 0, 0, mesh_resolution))  # Centro
        outer_points.append(gmsh.model.geo.addPoint(R_out, 0, 0, mesh_resolution))
        outer_points.append(gmsh.model.geo.addPoint(0, R_out, 0, mesh_resolution))
        outer_points.append(gmsh.model.geo.addPoint(-R_out, 0, 0, mesh_resolution))
        outer_points.append(gmsh.model.geo.addPoint(0, -R_out, 0, mesh_resolution))
        
        # Arcos del círculo exterior
        outer_arcs = []
        outer_arcs.append(gmsh.model.geo.addCircleArc(outer_points[1], outer_points[0], outer_points[2]))
        outer_arcs.append(gmsh.model.geo.addCircleArc(outer_points[2], outer_points[0], outer_points[3]))
        outer_arcs.append(gmsh.model.geo.addCircleArc(outer_points[3], outer_points[0], outer_points[4]))
        outer_arcs.append(gmsh.model.geo.addCircleArc(outer_points[4], outer_points[0], outer_points[1]))
        
        # Curva loop exterior
        outer_loop = gmsh.model.geo.addCurveLoop(outer_arcs)
        
        # Puntos para el círculo interior
        inner_points = []
        inner_points.append(gmsh.model.geo.addPoint(0, 0, 0, mesh_resolution))  # Mismo centro
        inner_points.append(gmsh.model.geo.addPoint(R_in, 0, 0, mesh_resolution))
        inner_points.append(gmsh.model.geo.addPoint(0, R_in, 0, mesh_resolution))
        inner_points.append(gmsh.model.geo.addPoint(-R_in, 0, 0, mesh_resolution))
        inner_points.append(gmsh.model.geo.addPoint(0, -R_in, 0, mesh_resolution))
        
        # Arcos del círculo interior
        inner_arcs = []
        inner_arcs.append(gmsh.model.geo.addCircleArc(inner_points[1], inner_points[0], inner_points[2]))
        inner_arcs.append(gmsh.model.geo.addCircleArc(inner_points[2], inner_points[0], inner_points[3]))
        inner_arcs.append(gmsh.model.geo.addCircleArc(inner_points[3], inner_points[0], inner_points[4]))
        inner_arcs.append(gmsh.model.geo.addCircleArc(inner_points[4], inner_points[0], inner_points[1]))
        
        # Curva loop interior
        inner_loop = gmsh.model.geo.addCurveLoop(inner_arcs)
        
        # Crear superficie del anillo (exterior - interior)
        surface = gmsh.model.geo.addPlaneSurface([outer_loop, inner_loop])
        
        # Sincronizar y generar malla
        gmsh.model.geo.synchronize()
        
        # Asignar nombres físicos para las fronteras
        gmsh.model.addPhysicalGroup(1, [outer_arcs[0], outer_arcs[1], outer_arcs[2], outer_arcs[3]], 1)
        gmsh.model.setPhysicalName(1, 1, "Gamma_out")
        
        gmsh.model.addPhysicalGroup(1, [inner_arcs[0], inner_arcs[1], inner_arcs[2], inner_arcs[3]], 2)
        gmsh.model.setPhysicalName(1, 2, "Gamma_in")
        
        gmsh.model.addPhysicalGroup(2, [surface], 3)
        gmsh.model.setPhysicalName(2, 3, "Omega")
        
        # Generar malla 2D
        gmsh.model.mesh.generate(2)
        
        # Crear directorio temporal para archivos intermedios
        temp_dir = tempfile.mkdtemp()
        msh_path = os.path.join(temp_dir, "annulus_mesh.msh")
        
        # Guardar malla en formato .msh
        gmsh.write(msh_path)
        
        # Convertir a formato XDMF para dolfinx
        convert_gmsh_to_xdmf(msh_path, filename, cell_type=CellType.triangle)
        
        # Limpiar archivo temporal
        if os.path.exists(msh_path):
            os.remove(msh_path)
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)
            
        print(f"✓ Malla de anillo creada: R_in={R_in}, R_out={R_out}")
        print(f"✓ Archivo guardado: {filename}")
        
        return filename
        
    except Exception as e:
        print(f"❌ Error creando malla: {e}")
        raise
    finally:
        # Asegurarse de que GMSH se cierra
        gmsh.finalize()

def convert_gmsh_to_xdmf(gmsh_file, xdmf_file, cell_type=CellType.triangle):
    """Convierte archivo .msh de GMSH a formato XDMF para dolfinx"""
    try:
        import meshio
    except ImportError:
        raise ImportError("meshio no está instalado. Ejecuta: pip install meshio")
    
    # Leer malla con meshio
    msh = meshio.read(gmsh_file)
    
    # Extraer células y puntos
    if cell_type == CellType.triangle:
        triangle_cells = []
        for cell in msh.cells:
            if cell.type == "triangle":
                triangle_cells.append(cell.data)
        
        if triangle_cells:
            triangle_cells = np.vstack(triangle_cells)
        else:
            raise ValueError("No se encontraron elementos triangulares en la malla")
        
        # Crear directorio si no existe
        os.makedirs(os.path.dirname(xdmf_file) if os.path.dirname(xdmf_file) else '.', exist_ok=True)
        
        # Crear malla meshio
        mesh_meshio = meshio.Mesh(
            points=msh.points[:, :2],  # Solo coordenadas x,y para 2D
            cells=[("triangle", triangle_cells)],
            cell_data={"name_to_read": [np.ones(triangle_cells.shape[0])]},
            field_data=msh.field_data
        )
        
        # Guardar en formato XDMF
        mesh_meshio.write(xdmf_file)
    else:
        raise ValueError("Tipo de celda no soportado")

def load_and_visualize_mesh(filename):
    """Carga y visualiza la malla generada"""
    comm = MPI.COMM_WORLD
    
    try:
        with io.XDMFFile(comm, filename, "r") as xdmf:
            domain = xdmf.read_mesh()
        
        print(f"=== INFORMACIÓN DE LA MALLA ===")
        print(f"Geometría: Anillo 2D")
        print(f"Dimensiones: {domain.geometry.x.shape}")
        print(f"Número de elementos: {domain.topology.index_map(2).size_global}")
        print(f"Número de vértices: {domain.geometry.x.shape[0]}")
        
        # Calcular radios reales
        coords = domain.geometry.x
        radii = np.sqrt(coords[:,0]**2 + coords[:,1]**2)
        print(f"Radio mínimo: {np.min(radii):.3f}")
        print(f"Radio máximo: {np.max(radii):.3f}")
        
        return domain
    except Exception as e:
        print(f"❌ Error cargando malla: {e}")
        return None

# VERSIÓN SIMPLIFICADA SIN GMSH (alternativa)
def create_simple_annulus_mesh(R_in=0.1, R_out=1.0, n_circumferential=32, n_radial=8):
    """
    Crea una malla simple de anillo sin dependencias externas
    """
    comm = MPI.COMM_WORLD
    
    # Crear malla rectangular y mapear a coordenadas polares
    n_cells = [n_circumferential, n_radial]
    domain = mesh.create_rectangle(comm, [np.array([0, R_in]), np.array([2*np.pi, R_out])], 
                                   n_cells, mesh.CellType.quadrilateral)
    
    # Transformar a coordenadas polares
    points = domain.geometry.x.copy()
    r = points[:, 1]  # Coordenada radial
    theta = points[:, 0]  # Coordenada angular
    
    # Convertir a coordenadas cartesianas
    points[:, 0] = r * np.cos(theta)
    points[:, 1] = r * np.sin(theta)
    
    # Actualizar coordenadas de la malla
    domain.geometry.x[:, :] = points[:, :]
    
    return domain

# PARÁMETROS DE LA GEOMETRÍA
R_in = 0.1    # Radio interno (Γ_in,0 - borde interno)
R_out = 1.0   # Radio externo (Γ_out,0 - borde externo)

if __name__ == "__main__":
    print("=== GENERANDOR MALLA DE ANILLO 2D ===")
    print(f"Parámetros: R_in={R_in}, R_out={R_out}")
    
    # Opción 1: Usar método simple (recomendado para evitar problemas con GMSH)
    try:
        print("\n--- Intentando método simple ---")
        domain_simple = create_simple_annulus_mesh(R_in, R_out)
        
        # Guardar malla simple
        simple_filename = "simple_annulus_mesh.xdmf"
        with io.XDMFFile(MPI.COMM_WORLD, simple_filename, "w") as xdmf:
            xdmf.write_mesh(domain_simple)
        
        print(f"✓ Malla simple guardada en: {simple_filename}")
        
        # Visualizar malla simple
        domain = load_and_visualize_mesh(simple_filename)
        
    except Exception as e:
        print(f"❌ Error con método simple: {e}")
        
        # Opción 2: Usar GMSH (puede fallar en entornos restringidos)
        try:
            print("\n--- Intentando con GMSH ---")
            mesh_filename = create_annulus_mesh(R_in, R_out, mesh_resolution=0.05)
            domain = load_and_visualize_mesh(mesh_filename)
        except Exception as e2:
            print(f"❌ Error con GMSH: {e2}")
            print("💡 Sugerencia: Usa el método simple o instala meshio: pip install meshio")
    
    print("\n" + "="*50)
    print("PROCESO COMPLETADO")
    print("="*50)

=== GENERANDOR MALLA DE ANILLO 2D ===
Parámetros: R_in=0.1, R_out=1.0

--- Intentando método simple ---
✓ Malla simple guardada en: simple_annulus_mesh.xdmf
=== INFORMACIÓN DE LA MALLA ===
Geometría: Anillo 2D
Dimensiones: (297, 3)
Número de elementos: 256
Número de vértices: 297
Radio mínimo: 0.000
Radio máximo: 1.307

PROCESO COMPLETADO


In [38]:
from mpi4py import MPI
from petsc4py import PETSc
import numpy as np
from basix.ufl import element, mixed_element
from dolfinx import fem, io, mesh
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from ufl import (Identity, TestFunctions, TrialFunctions, split, 
                 grad, det, tr, inner, derivative, dx, variable, inv, ln, TrialFunction)
import ufl
import os
import tempfile
import adios4dolfinx
import pickle

# Crear malla
domain = mesh.create_unit_square(MPI.COMM_WORLD, 32, 32, mesh.CellType.triangle)

# Crear directorio de resultados
results_dir = tempfile.mkdtemp(prefix="fem_results_")
print(f"Results will be saved in: {results_dir}")

# ELEMENTOS ROBUSTOS
k = 2
V_el = element("Lagrange", domain.basix_cell(), k, shape=(domain.geometry.dim,))
Q_el = element("Lagrange", domain.basix_cell(), k-1)
V_mixed = mixed_element([V_el, Q_el])
V = fem.functionspace(domain, V_mixed)

# PARÁMETROS 
mu = fem.Constant(domain, 1.0)
lmbda = fem.Constant(domain, 100.0)

# Función para la solución
u_p_ = fem.Function(V)
u, p = split(u_p_)

# Funciones test
v, q = TestFunctions(V)

# --- Neo-Hooke mixto (u, p) ---

d = domain.geometry.dim
I = Identity(d)

# Desplazamiento u y presión p ya definidos como Function en su espacio
# u_p_ = Function(W) con split(u_p_) -> (u, p)

F = ufl.variable(I + grad(u))
J = ufl.det(F)
C = F.T * F
Ic = ufl.tr(C)

# Parte isocórica (deviatoric) tipo Neo-Hooke modificado
psi_iso = (mu / 2) * (Ic - 2 - 2 * ufl.ln(J))

# Parte volumétrica en formulación mixta
psi_vol = -p * ufl.ln(J) + (1.0 / (2.0 * lmbda)) * p**2

psi = psi_iso + psi_vol

# Piola de 1ª especie
P = ufl.diff(psi, F)

# Forma débil: equilibrio + ecuación de estado (ln J - p/λ = 0)
G = (
    inner(P, grad(v)) * dx
    + (ufl.ln(J) - p / lmbda) * q * dx
)


# Jacobiano para Newton
du = TrialFunction(V)          # V: espacio de desplazamientos dentro de W
J_form = derivative(G, u_p_, du)


# CONDICIONES DE BORDE
def bottom(x):
    return np.isclose(x[1], 0.0)

def top(x):
    return np.isclose(x[1], 1.0)

bottom_facets = mesh.locate_entities_boundary(domain, domain.topology.dim-1, bottom)
top_facets = mesh.locate_entities_boundary(domain, domain.topology.dim-1, top)

# Fija u_x = 0 y u_y = 0 en la cara inferior
V0_bottom, _ = V.sub(0).collapse()
bottom_dofs = fem.locate_dofs_topological((V.sub(0), V0_bottom), domain.topology.dim-1, bottom_facets)
u_bottom = fem.Function(V0_bottom)
u_bottom.x.array[:] = 0.0
bc_bottom = fem.dirichletbc(u_bottom, bottom_dofs, V.sub(0))

# BC superior - SOLO en componente Y
V0_y, _ = V.sub(0).sub(1).collapse()
top_dofs_y = fem.locate_dofs_topological((V.sub(0).sub(1), V0_y), domain.topology.dim-1, top_facets)

u_top_val = fem.Function(V0_y)
u_top_val.x.array[:] = 0.0
bc_top = fem.dirichletbc(u_top_val, top_dofs_y, V.sub(0).sub(1))

bcs = [bc_bottom, bc_top]
print("✓ BCs corregidas: Cara inferior fija, Cara superior desliza en X.")

# SOLVER CONFIGURADO
problem = NonlinearProblem(G, u_p_, bcs=bcs, J=J_form)
solver = NewtonSolver(MPI.COMM_WORLD, problem)
solver.atol = 1e-8
solver.rtol = 1e-8
solver.max_it = 50
solver.convergence_criterion = "incremental"

ksp = solver.krylov_solver
opts = PETSc.Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "gmres"
opts[f"{option_prefix}pc_type"] = "lu"
opts[f"{option_prefix}pc_factor_mat_solver_type"] = "mumps"
ksp.setFromOptions()

# CARGA ADAPTATIVA
def adaptive_load_increments(initial_disp, target_disp, max_steps=50):
    increments = []
    current = initial_disp
    
    while current < target_disp:
        increments.append(current)
        if current < target_disp * 0.1:
            step = target_disp / 30
        elif current < target_disp * 0.5:
            step = target_disp / 20
        else:
            step = target_disp / 15
            
        current += step
        if current > target_disp:
            current = target_disp
    
    increments.append(target_disp)
    return np.unique(increments)

initial_displacement = 0.01
target_displacement = 0.1
max_steps = 100

load_values = adaptive_load_increments(initial_displacement, target_displacement, max_steps)
num_steps = len(load_values) - 1

print(f"Estrategia de carga adaptativa:")
print(f"   - Desplazamiento objetivo: {target_displacement:.3f}")
print(f"   - Número de pasos: {num_steps}")

# Preparar espacios de visualización P1
V_vis = fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim,)))
Q_vis = fem.functionspace(domain, ("Lagrange", 1))

print("\n=== Iniciando simulación directa ===")

convergence_history = []
all_converged = True

for i, disp in enumerate(load_values[1:]):
    step_size = disp - load_values[i]
    print(f"Paso {i+1}/{num_steps}, Desplazamiento = {disp:.4f} (Δ = {step_size:.4f})")
    
    u_top_val.x.array[:] = -disp
    
    try:
        num_its, converged = solver.solve(u_p_)
        
        if converged:
            print(f"   ✓ Converged in {num_its} iterations")
            convergence_history.append(num_its)
            
            u_sol = u_p_.sub(0).collapse()
            u_array = u_sol.x.array.reshape(-1, domain.geometry.dim)
            max_deformation_x = np.max(np.abs(u_array[:, 0]))
            max_deformation_y = np.max(np.abs(u_array[:, 1]))
            print(f"   → Deformación máxima Y: {max_deformation_y:.4f}")
            print(f"   → Deformación máxima X (abombamiento): {max_deformation_x:.4f}")
            
        else:
            print(f"   ✗ FAILED to converge after {num_its} iterations")
            all_converged = False
            break
            
    except Exception as e:
        print(f"   ✗ ERROR: {e}")
        all_converged = False
        break

print("\n=== Guardando resultados ===")

if all_converged:
    
    print("--- Guardando datos para problema inverso (Alta Fidelidad) ---")
    
    u_sol = u_p_.sub(0).collapse()
    p_sol = u_p_.sub(1).collapse()
    u_sol.name = "Displacement"
    p_sol.name = "Pressure"

    # Guardar malla
    mesh_file = os.path.join(results_dir, "simulation_mesh.bp")
    adios4dolfinx.write_mesh(mesh_file, domain, engine="BP4")

    # Guardar funciones de alta fidelidad
    disp_file = os.path.join(results_dir, "displacement.bp")
    pres_file = os.path.join(results_dir, "pressure.bp")
    
    adios4dolfinx.write_function(disp_file, u_sol, engine="BP4")
    adios4dolfinx.write_function(pres_file, p_sol, engine="BP4")
    
    print(f"Datos para inverso guardados:")
    print(f" - {os.path.basename(mesh_file)} (Malla original)")
    print(f" - {os.path.basename(disp_file)} (Desplazamiento k={k})")
    print(f" - {os.path.basename(pres_file)} (Presión k={k-1})")

    # --- 2. DATOS PARA VISUALIZACIÓN EN XDMF (P1) ---
    print("\n--- Guardando resultados en XDMF para visualización (P1) ---")
    
    # Interpolar a P1 para XDMF
    u_vis = fem.Function(V_vis)
    u_vis.name = "Displacement"
    u_vis.interpolate(u_sol)
    
    p_vis = fem.Function(Q_vis)
    p_vis.name = "Pressure"
    p_vis.interpolate(p_sol)
    
    # Guardar en XDMF (malla original)
    xdmf_file = io.XDMFFile(domain.comm, os.path.join(results_dir, "simulation_results.xdmf"), "w")
    xdmf_file.write_mesh(domain)
    xdmf_file.write_function(u_vis, 0.0)
    xdmf_file.write_function(p_vis, 0.0)
    xdmf_file.close()
    print("✓ XDMF guardado: simulation_results.xdmf")
    
    print("\n--- Creando malla deformada para visualización ---")
    
    # Calcular nuevas coordenadas
    original_coords = domain.geometry.x
    displacement_at_vertices = u_vis.x.array.reshape((-1, domain.geometry.dim))
    
    displacement_3d = np.zeros_like(original_coords)
    displacement_3d[:, :domain.geometry.dim] = displacement_at_vertices
    new_coords = original_coords + displacement_3d
    
    # Crear malla deformada
    cells_array = domain.topology.connectivity(domain.topology.dim, 0).array.reshape((-1, domain.topology.dim + 1))
    cells_array = np.asarray(cells_array, dtype=np.int64)
    deformed_mesh = mesh.create_mesh(domain.comm, cells_array, new_coords[:, :domain.geometry.dim], domain.ufl_domain())
    
    # Crear funciones en malla deformada
    V_vis_deformed = fem.functionspace(deformed_mesh, ("Lagrange", 1, (deformed_mesh.geometry.dim,)))
    Q_vis_deformed = fem.functionspace(deformed_mesh, ("Lagrange", 1))
    
    u_vis_def = fem.Function(V_vis_deformed)
    u_vis_def.name = "Displacement"
    u_vis_def.x.array[:] = u_vis.x.array
    
    p_vis_def = fem.Function(Q_vis_deformed)
    p_vis_def.name = "Pressure"
    p_vis_def.x.array[:] = p_vis.x.array
    
    # Guardar malla deformada en XDMF
    xdmf_deformed = io.XDMFFile(deformed_mesh.comm, os.path.join(results_dir, "deformed_results.xdmf"), "w")
    xdmf_deformed.write_mesh(deformed_mesh)
    xdmf_deformed.write_function(u_vis_def, 0.0)
    xdmf_deformed.write_function(p_vis_def, 0.0)
    xdmf_deformed.close()
    print("✓ XDMF deformado guardado: deformed_results.xdmf")
    
    # También guardar con adios4dolfinx
    vis_disp_file = os.path.join(results_dir, "VISUALIZATION_displacement.bp")
    vis_pres_file = os.path.join(results_dir, "VISUALIZATION_pressure.bp")
    
    adios4dolfinx.write_function(vis_disp_file, u_vis_def, engine="BP4")
    adios4dolfinx.write_function(vis_pres_file, p_vis_def, engine="BP4")
    print("✓ Archivos BP4 guardados para ParaView")

    print(f"\n✅ Simulación completada exitosamente")
    print(f"📂 Resultados guardados en: {results_dir}")
    print(f"\n=== Instrucciones de Visualización ===")
    print("\n📊 OPCIÓN 1 - ParaView con XDMF:")
    print("  1. Abre 'simulation_results.xdmf'")
    print("  2. Aplica 'Warp By Vector' usando 'Displacement'")
    print("  3. O abre 'deformed_results.xdmf' (ya deformada)")
    print("\n📊 OPCIÓN 2 - ParaView con BP4:")
    print("  1. Abre 'VISUALIZATION_displacement.bp'")
    print("  2. La malla aparece directamente deformada")
    print("\n🔬 Para problema inverso:")
    print("  - Usa 'simulation_mesh.bp'")
    print("  - Usa 'displacement.bp' (k=2)")
    print("  - Usa 'pressure.bp' (k=1)")

else:
    print(f"✗ Simulación no convergente. No se guardaron resultados en: {results_dir}")

Results will be saved in: /tmp/fem_results_6jlbt10y
✓ BCs corregidas: Cara inferior fija, Cara superior desliza en X.
Estrategia de carga adaptativa:
   - Desplazamiento objetivo: 0.100
   - Número de pasos: 17

=== Iniciando simulación directa ===
Paso 1/17, Desplazamiento = 0.0133 (Δ = 0.0033)
   ✓ Converged in 4 iterations
   → Deformación máxima Y: 0.0133
   → Deformación máxima X (abombamiento): 0.0080
Paso 2/17, Desplazamiento = 0.0183 (Δ = 0.0050)
   ✓ Converged in 4 iterations
   → Deformación máxima Y: 0.0183
   → Deformación máxima X (abombamiento): 0.0111
Paso 3/17, Desplazamiento = 0.0233 (Δ = 0.0050)
   ✓ Converged in 4 iterations
   → Deformación máxima Y: 0.0233
   → Deformación máxima X (abombamiento): 0.0142
Paso 4/17, Desplazamiento = 0.0283 (Δ = 0.0050)
   ✓ Converged in 4 iterations
   → Deformación máxima Y: 0.0283
   → Deformación máxima X (abombamiento): 0.0173
Paso 5/17, Desplazamiento = 0.0333 (Δ = 0.0050)
   ✓ Converged in 4 iterations
   → Deformación máxima 

# Mecánica Pulmonar: Análisis con Surfactante

## 2. Con Surfactante

### Funcional de Energía con Contribución Superficial

Se añade a la funcional anterior una contribución de energía de superficie en la interfaz fluido–sólido (pared alveolar) debida al surfactante pulmonar:

$$\Pi_{\text{surf}}[\chi] = \Pi_0[\chi] + \int_{\Gamma_{\text{alv},0}} \gamma(J_s)\, dA_0$$

#### Nuevos Parámetros:

| Símbolo | Descripción |
|---------|-------------|
| $\gamma(J_s)$ | Tensión superficial efectiva (dependiente del stretch superficial) |
| $J_s$ | Determinante tangencial del gradiente de deformación en $\Gamma_{\text{alv},0}$ (mide el estiramiento de la superficie) |
| $A$ | Área actual de la interfaz |

**Característica clave:** La tensión superficial es **variable**, dependiendo del área (o del stretch) de la interfaz.

---

## 3. Ecuaciones de Equilibrio con Surfactante

### Ecuación de Equilibrio en el Volumen

La ecuación en el volumen **permanece igual**:

$$\text{Div}_X\,P(F) + \rho_0\,\mathbf{g} = \mathbf{0} \quad\text{en } \Omega_0$$

### Condición de Borde Modificada en la Interfaz Alveolar

La variación de $\Pi_{\text{surf}}$ produce una condición de borde interna **modificada**:

$$P(F)\,\mathbf{N} + \mathbf{t}_{\text{surf}}(F) = -p_{\text{alv}}\, J\,F^{-T}\mathbf{N} \quad\text{en } \Gamma_{\text{alv},0}$$

donde:
- $\mathbf{t}_{\text{surf}}(F)$ es la **tracción adicional debida al surfactante**
- Esta tracción surge de la variación de la energía superficial

---

## 4. Aproximaciones Especiales

### 4.1 Tensión Superficial Constante (Ley de Laplace Clásica)

Si $\gamma$ es **constante** (independiente del área), la tracción superficial toma la forma típica de la **ley de Young-Laplace**:

$$\mathbf{t}_{\text{surf}} \sim 2\gamma\,H\,\mathbf{n}$$

donde:
- $\gamma$ es la tensión superficial constante
- $H$ es la **curvatura media** de la interfaz
- $\mathbf{n}$ es la normal en la configuración actual
- El factor $2H$ representa la suma de curvaturas principales

**Interpretación física:** La presencia de una interfaz curva con tensión superficial produce una tracción normal que depende de la geometría local.

### 4.2 Tensión Superficial Dependiente del Área (Modelo Realista)

Cuando $\gamma$ depende del estiramiento superficial $J_s$:

$$\gamma = \gamma(J_s)$$

La tracción superficial contiene términos adicionales proporcionales a $\frac{d\gamma}{dJ_s}$. Conceptualmente:

$$\mathbf{t}_{\text{surf}} = 2\gamma(J_s)\,H\,\mathbf{n} + \text{términos adicionales con } \frac{d\gamma}{dJ_s}$$

**Característica importante:** El surfactante pulmonar real se comporta como un **resorte que se "ablanda"** cuando la interfaz se estira (es decir, $\frac{d\gamma}{dJ_s} < 0$). Esto reduce la resistencia mecánica del pulmón durante la inspiración.

---

## 5. Solución: Configuración con Surfactante

La configuración de equilibrio con surfactante $\chi_{\text{surf}}$ es la **solución del mismo problema de equilibrio** pero con la **condición de borde modificada**:

$$\begin{cases}
\text{Div}_X\,P(F) + \rho_0\,\mathbf{g} = \mathbf{0} & \text{en } \Omega_0 \\
P(F)\,\mathbf{N} + \mathbf{t}_{\text{surf}}(F) = -p_{\text{alv}}\, J\,F^{-T}\mathbf{N} & \text{en } \Gamma_{\text{alv},0} \\
\cdots & \text{(otras condiciones de borde)}
\end{cases}$$

### Diferencia Crucial

Para una **misma presión alveolar** $p_{\text{alv}}$, las configuraciones:

$$\chi_{\text{surf}} \quad\text{y}\quad \chi_0$$

**no coinciden en general**. La diferencia es atribuible al efecto mecánico del surfactante:
- El surfactante modifica la **deformación local** del tejido
- Cambia la **rigidez efectiva** de las paredes alveolares
- Afecta la **distribución de esfuerzos** en toda la estructura

---

## 6. Implicaciones Fisiológicas

| Aspecto | Sin Surfactante | Con Surfactante |
|--------|-----------------|-----------------|
| **Tensión superficial** | No existe | Variable con el área |
| **Tracción en frontera** | Solo presión interna | Presión + contribución superficial |
| **Rigidez efectiva** | Mayor (sin "ablandamiento") | Menor (especialmente al estirar) |
| **Compliance pulmonar** | Reducida | Aumentada |
| **Energía requerida** | Mayor | Menor (efecto protector) |
| **Inestabilidad alveolar** | Posible | Prevenida |

---

## 7. Notas Adicionales

- El surfactante pulmonar es una mezcla compleja de lípidos y proteínas que en modelos continuum se simplifica como una densidad de energía superficial
- La dependencia $\gamma(J_s)$ captura el comportamiento de "phase transition" del surfactante durante ciclos de compresión-expansión
- La comparación entre $\chi_0$ y $\chi_{\text{surf}}$ permite cuantificar la **contribución mecánica del surfactante** al funcionamiento pulmonar normal

Simulacion Multiescala